## This notebook contains ALL 8 matching methods:

### String Distance Methods (Cohen et al.):
1. **Jaro-Winkler** - Character-level similarity with prefix boost
2. **Levenshtein** - Edit distance (insertions, deletions, substitutions)
3. **Monge-Elkan** - Token-level matching with Jaro-Winkler base

### Token-Based Methods (Cohen et al.):
4. **TF-IDF** - Traditional cosine similarity on term frequency vectors
5. **Soft-TFIDF** - Fuzzy TF-IDF with approximate token matching (θ=0.9)

### Embedding Methods:
6. **SentenceTransformer** - Local embeddings (all-MiniLM-L6-v2)
7. **OpenAI Embeddings** - API embeddings (text-embedding-3-small)

### LLM Method:
8. **GPT-4o-mini** - LLM-based matching with blocking strategy

## Installation & Setup

In [1]:
!pip install jellyfish py_stringmatching sentence-transformers openai tqdm scikit-learn python-dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [10]:
# 📚 IMPORTS
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm
from typing import Tuple, Optional, Set, Dict
import warnings
warnings.filterwarnings('ignore')

# String matching
import jellyfish
import py_stringmatching as sm

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Embeddings
from sentence_transformers import SentenceTransformer

# OpenAI
from openai import OpenAI
from dotenv import load_dotenv

# Display
from IPython.display import display

print("Libraries Imported Successfully!")

Libraries Imported Successfully!


## Configuration

- **TEST_MODE = True**: Uses only 100 records 
- **TEST_MODE = False**: Uses full dataset

In [11]:
# TEST MODE: Set to True for quick test (100 records)
TEST_MODE = False  # 

# Dataset selection
DATASET_NAME = 'abt-buy'  # Options: 'abt-buy', 'amazon-google', 'dblp-acm', 'dblp-scholar'


CONFIG = {
    'dataset_name': DATASET_NAME,
    
    # Abt-Buy paths (using ../ to go up from notebooks/ folder)
    'dataset_a': '../data/abt-buy/Abt.csv',
    'dataset_b': '../data/abt-buy/Buy.csv',
    'dataset_a_ciphered': '../data/abt-buy/Abt_ciphered.csv',
    'dataset_b_ciphered': '../data/abt-buy/Buy_ciphered.csv',
    'dataset_a_scrambled': '../data/abt-buy/Abt_scrambled.csv',
    'dataset_b_scrambled': '../data/abt-buy/Buy_scrambled.csv',
    'ground_truth': '../data/abt-buy/abt_buy_perfectMapping.csv',
    
    # Column names for Abt-Buy
    'id_col_a': 'id',
    'id_col_b': 'id',
    'text_cols': ['name', 'description'],
    'gt_col_a': 'idAbt',
    'gt_col_b': 'idBuy',
}

# Amazon-Google:
# CONFIG['text_cols'] = ['title', 'description', 'manufacturer']
# CONFIG['gt_col_a'] = 'idAmazon'
# CONFIG['gt_col_b'] = 'idGoogleProducts'

# DBLP-ACM or DBLP-Scholar:
# CONFIG['text_cols'] = ['title', 'authors', 'venue']
# CONFIG['gt_col_a'] = 'idDBLP'
# CONFIG['gt_col_b'] = 'idACM'  # or 'idScholar'


# OpenAI API Key
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if OPENAI_API_KEY:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("OpenAI API key loaded")
else:
    print("No OpenAI API key - OpenAI methods will be skipped")

# Display Settings
print("\n" + "="*60)
if TEST_MODE:
    print("TEST MODE ENABLED")
    print("- Using only 100 records per dataset")
else:
    print("FULL MODE ENABLED")
    print("- Using complete datasets")
print("="*60)
print(f"Dataset: {CONFIG['dataset_name']}")
print("="*60)

OpenAI API key loaded

FULL MODE ENABLED
- Using complete datasets
Dataset: abt-buy


## Load Data

In [12]:
def load_csv_with_encoding(filepath):
    """
    Load CSV with automatic encoding detection.
    Tries UTF-8, Latin-1, Windows-1252, and ISO-8859-1.
    """
    encodings = ['utf-8', 'latin-1', 'windows-1252', 'iso-8859-1']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(filepath, encoding=encoding)
            print(f"   Loaded with {encoding} encoding")
            return df
        except (UnicodeDecodeError, UnicodeError):
            continue
    
    # If all fail, raise error
    raise ValueError(f"Could not read {filepath} with any encoding: {encodings}")

print("Loading datasets with auto-encoding detection...\n")

try:
    # Load original datasets
    print("Loading Original Dataset A...")
    df_a = load_csv_with_encoding(CONFIG['dataset_a'])
    print(f"Dataset A: {len(df_a):,} records\n")
    
    print("Loading Original Dataset B...")
    df_b = load_csv_with_encoding(CONFIG['dataset_b'])
    print(f"Dataset B: {len(df_b):,} records\n")
    
    # Load ciphered datasets
    print("Loading Ciphered Dataset A...")
    df_a_ciphered = load_csv_with_encoding(CONFIG['dataset_a_ciphered'])
    print(f"Ciphered A: {len(df_a_ciphered):,} records\n")
    
    print("Loading Ciphered Dataset B...")
    df_b_ciphered = load_csv_with_encoding(CONFIG['dataset_b_ciphered'])
    print(f"Ciphered B: {len(df_b_ciphered):,} records\n")
    
    # Load scrambled datasets
    print("Loading Scrambled Dataset A...")
    df_a_scrambled = load_csv_with_encoding(CONFIG['dataset_a_scrambled'])
    print(f"Scrambled A: {len(df_a_scrambled):,} records\n")
    
    print("Loading Scrambled Dataset B...")
    df_b_scrambled = load_csv_with_encoding(CONFIG['dataset_b_scrambled'])
    print(f"Scrambled B: {len(df_b_scrambled):,} records\n")
    
    # Load ground truth
    print("Loading Ground Truth...")
    ground_truth = load_csv_with_encoding(CONFIG['ground_truth'])
    print(f"\nGround Truth: {len(ground_truth):,} known matches")
    
except FileNotFoundError as e:
    print(f"\nERROR: Could not find file!")  
    raise

# ============================================
# APPLY TEST MODE
# ============================================

if TEST_MODE:
    print("\n" + "="*60)
    print("TEST MODE: Reducing to 100 records per dataset")
    print("="*60)
    
    df_a = df_a.head(100)
    df_b = df_b.head(100)
    df_a_ciphered = df_a_ciphered.head(100)
    df_b_ciphered = df_b_ciphered.head(100)
    df_a_scrambled = df_a_scrambled.head(100)
    df_b_scrambled = df_b_scrambled.head(100)
    
    print(f" Reduced to {len(df_a)} × {len(df_b)} = {len(df_a) * len(df_b):,} comparisons")
    print(f"   (Original would be {1081 * 1092:,} comparisons)")
    print("="*60)

# Preview
print("\nData Preview:")
display(df_a.head(5))

Loading datasets with auto-encoding detection...

Loading Original Dataset A...
   Loaded with latin-1 encoding
Dataset A: 1,081 records

Loading Original Dataset B...
   Loaded with utf-8 encoding
Dataset B: 1,092 records

Loading Ciphered Dataset A...
   Loaded with latin-1 encoding
Ciphered A: 1,081 records

Loading Ciphered Dataset B...
   Loaded with utf-8 encoding
Ciphered B: 1,092 records

Loading Scrambled Dataset A...
   Loaded with latin-1 encoding
Scrambled A: 1,081 records

Loading Scrambled Dataset B...
   Loaded with utf-8 encoding
Scrambled B: 1,092 records

Loading Ground Truth...
   Loaded with utf-8 encoding

Ground Truth: 1,097 known matches

Data Preview:


,id,name,description,price
0,552,Sony Turntable - PSLX350H,Sony Turntable - PSLX350H/ Belt Drive System/ ...,NaN
1,580,Bose Acoustimass 5 Series III Speaker System -...,Bose Acoustimass 5 Series III Speaker System -...,$399.00
2,4696,Sony Switcher - SBV40S,Sony Switcher - SBV40S/ Eliminates Disconnecti...,$49.00
3,5644,Sony 5 Disc CD Player - CDPCE375,Sony 5 Disc CD Player- CDPCE375/ 5 Disc Change...,NaN
4,6284,Bose 27028 161 Bookshelf Pair Speakers In Whit...,Bose 161 Bookshelf Speakers In White - 161WH/ ...,$158.00


## Text Preprocessing

In [13]:
""" 
1. Handle missing values
2. Convert to lowercase
3. Combine multiple columns into one 'text' column
"""

def preprocess_text(df, text_columns):
    """
    Preprocess text for matching.
    """
    for col in text_columns:
        df[col] = df[col].fillna('')
    
    df['text'] = df[text_columns].apply(
        lambda row: ' '.join(row.values.astype(str)).lower().strip(),
        axis=1
    )
    
    return df

print("Preprocessing...\n")

df_a = preprocess_text(df_a, CONFIG['text_cols'])
df_b = preprocess_text(df_b, CONFIG['text_cols'])
df_a_ciphered = preprocess_text(df_a_ciphered, CONFIG['text_cols'])
df_b_ciphered = preprocess_text(df_b_ciphered, CONFIG['text_cols'])
df_a_scrambled = preprocess_text(df_a_scrambled, CONFIG['text_cols'])
df_b_scrambled = preprocess_text(df_b_scrambled, CONFIG['text_cols'])

print("All datasets preprocessed")

Preprocessing...

All datasets preprocessed


## Build Ground Truth Set

In [14]:
true_matches = set(
    zip(
        ground_truth[CONFIG['gt_col_a']].astype(str),
        ground_truth[CONFIG['gt_col_b']].astype(str)
    )
)

print(f"Ground truth: {len(true_matches):,} true matches")

Ground truth: 1,097 true matches


## Implementing Matching Methods

In [17]:
# ==================================================
# METHOD 1: Jaro-Winkler
# ==================================================

def jaro_winkler_similarity(str1, str2):
    """Character-level similarity with prefix boost."""
    return jellyfish.jaro_winkler_similarity(str1, str2)

print("Jaro-Winkler ready")

# ==================================================
# METHOD 2: Levenshtein
# ==================================================

def levenshtein_similarity(str1, str2):
    """Edit distance normalized to 0-1."""
    distance = jellyfish.levenshtein_distance(str1, str2)
    max_len = max(len(str1), len(str2))
    if max_len == 0:
        return 1.0
    return 1.0 - (distance / max_len)

print("Levenshtein ready")

# ==================================================
# METHOD 3: Monge-Elkan
# ==================================================

def monge_elkan_similarity(str1, str2):
    """Token-level matching with Jaro-Winkler base."""
    tokens_a = str1.split()
    tokens_b = str2.split()
    
    if len(tokens_a) == 0 or len(tokens_b) == 0:
        return 0.0
    
    sum_best = 0.0
    for token_a in tokens_a:
        best = max([jaro_winkler_similarity(token_a, token_b) for token_b in tokens_b])
        sum_best += best
    
    return sum_best / len(tokens_a)

print("Monge-Elkan ready")

# ==================================================
# METHOD 4: Soft-TFIDF
# ==================================================

def soft_tfidf_similarity(str1, str2, theta=0.9):
    """
    Soft-TFIDF with Jaro-Winkler and θ=0.9 (Cohen paper).
    Simplified version - uses token matching.
    """
    tokens_a = str1.split()
    tokens_b = str2.split()
    
    if len(tokens_a) == 0 or len(tokens_b) == 0:
        return 0.0
    
    score = 0.0
    for token_a in tokens_a:
        best_match = max([jaro_winkler_similarity(token_a, token_b) for token_b in tokens_b])
        if best_match >= theta:
            score += best_match
    
    return score / len(tokens_a) if len(tokens_a) > 0 else 0.0

print("Soft-TFIDF ready")


# ==================================================
# METHOD 5: TF-IDF
# ==================================================

def tfidf_matching(df_a, df_b, threshold, id_col_a, id_col_b):
    """
    TF-IDF cosine similarity with Top-1 matching.
    """
    vectorizer = TfidfVectorizer()
    all_texts = pd.concat([df_a['text'], df_b['text']])
    vectorizer.fit(all_texts)
    
    vectors_a = vectorizer.transform(df_a['text'])
    vectors_b = vectorizer.transform(df_b['text'])
    similarity_matrix = cosine_similarity(vectors_a, vectors_b)
    
    predicted_matches = set()
    for idx_a in range(len(df_a)):
        best_idx_b = np.argmax(similarity_matrix[idx_a])
        best_sim = similarity_matrix[idx_a, best_idx_b]
        
        if best_sim >= threshold:
            id_a = str(df_a.iloc[idx_a][id_col_a])
            id_b = str(df_b.iloc[best_idx_b][id_col_b])
            predicted_matches.add((id_a, id_b))
    
    return predicted_matches

print("TF-IDF ready")

# ==================================================
# METHOD 6: SentenceTransformer
# ==================================================

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

def sentence_transformer_matching(df_a, df_b, threshold, id_col_a, id_col_b):
    """
    SentenceTransformer embedding matching.
    """
    embeddings_a = sentence_model.encode(df_a['text'].tolist(), show_progress_bar=False)
    embeddings_b = sentence_model.encode(df_b['text'].tolist(), show_progress_bar=False)
    similarity_matrix = cosine_similarity(embeddings_a, embeddings_b)
    
    predicted_matches = set()
    for idx_a in range(len(df_a)):
        best_idx_b = np.argmax(similarity_matrix[idx_a])
        best_sim = similarity_matrix[idx_a, best_idx_b]
        
        if best_sim >= threshold:
            id_a = str(df_a.iloc[idx_a][id_col_a])
            id_b = str(df_b.iloc[best_idx_b][id_col_b])
            predicted_matches.add((id_a, id_b))
    
    return predicted_matches

print("Sentence Transformer matching ready")

# ==================================================
# METHOD 7: OpenAI Embedding Method
# ==================================================

if OPENAI_API_KEY:
    
    def get_openai_embeddings(texts, model="text-embedding-3-small"):
        """Get OpenAI embeddings in batches."""
        embeddings = []
        batch_size = 100
        
        for i in tqdm(range(0, len(texts), batch_size), desc="OpenAI embeddings"):
            batch = texts[i:i+batch_size]
            response = client.embeddings.create(model=model, input=batch)
            batch_embeddings = [item.embedding for item in response.data]
            embeddings.extend(batch_embeddings)
            time.sleep(0.1)
        
        return np.array(embeddings)
    
    def openai_embedding_matching(df_a, df_b, threshold, id_col_a, id_col_b):
        """OpenAI embedding matching."""
        embeddings_a = get_openai_embeddings(df_a['text'].tolist())
        embeddings_b = get_openai_embeddings(df_b['text'].tolist())
        similarity_matrix = cosine_similarity(embeddings_a, embeddings_b)
        
        predicted_matches = set()
        for idx_a in range(len(df_a)):
            best_idx_b = np.argmax(similarity_matrix[idx_a])
            best_sim = similarity_matrix[idx_a, best_idx_b]
            
            if best_sim >= threshold:
                id_a = str(df_a.iloc[idx_a][id_col_a])
                id_b = str(df_b.iloc[best_idx_b][id_col_b])
                predicted_matches.add((id_a, id_b))
        
        return predicted_matches
    
    print("OpenaAI Embedding matching ready")
    
# ==================================================

# ==================================================
# METHOD 8: LLM Matching Method
# ==================================================

def jaro_winkler_similarity(s1: str, s2: str) -> float:
    """Calculate Jaro-Winkler similarity between two strings."""
    if not s1 or not s2:
        return 0.0
    
    s1, s2 = s1.lower(), s2.lower()
    
    if s1 == s2:
        return 1.0
    
    len1, len2 = len(s1), len(s2)
    max_dist = max(len1, len2) // 2 - 1
    
    if max_dist < 0:
        max_dist = 0
    
    s1_matches = [False] * len1
    s2_matches = [False] * len2
    matches = 0
    
    for i in range(len1):
        start = max(0, i - max_dist)
        end = min(i + max_dist + 1, len2)
        
        for j in range(start, end):
            if s2_matches[j] or s1[i] != s2[j]:
                continue
            s1_matches[i] = True
            s2_matches[j] = True
            matches += 1
            break
    
    if matches == 0:
        return 0.0
    
    transpositions = 0
    k = 0
    for i in range(len1):
        if not s1_matches[i]:
            continue
        while not s2_matches[k]:
            k += 1
        if s1[i] != s2[k]:
            transpositions += 1
        k += 1
    
    jaro = (matches / len1 + matches / len2 + (matches - transpositions / 2) / matches) / 3
    
    prefix = 0
    for i in range(min(len1, len2, 4)):
        if s1[i] == s2[i]:
            prefix += 1
        else:
            break
    
    return jaro + prefix * 0.1 * (1 - jaro)



def llm_match_improved(record_a: pd.Series, 
                       df_b: pd.DataFrame, 
                       id_col_b: str,
                       client,
                       top_k: int = 20,
                       blocking_threshold: float = 0.1,
                       max_text_length: int = 500) -> Tuple[Optional[str], float, float, str]:
    """
    Improved LLM matching with optimizations.
    
    Args:
        record_a: Record to match from dataset A
        df_b: Dataset B to search for matches
        id_col_b: ID column name in dataset B
        client: OpenAI client
        top_k: Number of candidates to show LLM (default: 20)
        blocking_threshold: Minimum Jaro-Winkler to consider (default: 0.1)
        max_text_length: Maximum text length to send to LLM (default: 500, 0 = no limit)
    
    Returns:
        Tuple of (matched_id, confidence_score, cost, reasoning)
    """
    text_a = record_a['text']
    
    # IMPROVEMENT 1: More permissive blocking (0.1 instead of 0.3)
    # Find top-K candidates using Jaro-Winkler
    similarities = []
    for idx_b, row_b in df_b.iterrows():
        sim = jaro_winkler_similarity(text_a, row_b['text'])
        similarities.append((idx_b, sim, row_b[id_col_b], row_b['text']))
    
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # IMPROVEMENT 2: Increased top-k (20 instead of 10)
    top_candidates = similarities[:top_k]
    
    # More permissive filtering
    if len(top_candidates) == 0 or top_candidates[0][1] < blocking_threshold:
        return None, 0.0, 0.0, f"Best JW similarity {top_candidates[0][1] if top_candidates else 0:.3f} below threshold {blocking_threshold}"
    
    # IMPROVEMENT 3: No truncation (or higher limit)
    # Build candidate list with full text (or limited to max_text_length)
    candidates_list = []
    for i, cand in enumerate(top_candidates):
        text = cand[3]
        if max_text_length > 0 and len(text) > max_text_length:
            text = text[:max_text_length] + "..."
        candidates_list.append(f"[{i+1}] {text}")
    
    candidates_text = "\n\n".join(candidates_list)
    
    # Truncate query text if needed
    query_text = text_a
    if max_text_length > 0 and len(query_text) > max_text_length:
        query_text = query_text[:max_text_length] + "..."
    
    # IMPROVEMENT 4: Enhanced prompt with context and guidance
    prompt = f"""You are an expert at entity matching for product databases. Your task is to determine if any candidate record matches the query record, considering they come from different sources with varying descriptions.

QUERY RECORD:
{query_text}

CANDIDATE RECORDS:
{candidates_text}

MATCHING GUIDELINES:
- Products are the SAME if they refer to the same model/item, even with different descriptions
- Consider: brand names, model numbers, key specifications
- Account for: abbreviations (e.g., "PS5" = "PlayStation 5"), different word orders, extra/missing details
- Be strict about: core product identity (don't match PS5 with PS4, iPad Pro 11" with iPad Pro 12.9")
- Different bundles/configurations of the same base product may or may not match depending on significance

RESPONSE FORMAT:
If a match exists, respond with:
Match: [number]
Confidence: [0.0 to 1.0]
Reasoning: [brief explanation]

If no match exists, respond with:
Match: 0
Confidence: [0.0 to 1.0]
Reasoning: [why no candidates match]

Your response:"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        
        answer = response.choices[0].message.content.strip()
        tokens = response.usage.total_tokens
        cost = (tokens / 1_000_000) * 0.15
        
        # IMPROVEMENT 5: Parse confidence score
        match_num = 0
        confidence = 0.0
        reasoning = answer
        
        # Parse the response
        lines = answer.split('\n')
        for line in lines:
            line = line.strip()
            if line.startswith('Match:'):
                try:
                    match_num = int(''.join(c for c in line.split('Match:')[1] if c.isdigit()))
                except:
                    pass
            elif line.startswith('Confidence:'):
                try:
                    conf_str = line.split('Confidence:')[1].strip()
                    confidence = float(''.join(c for c in conf_str if c.isdigit() or c == '.'))
                    # Ensure confidence is between 0 and 1
                    if confidence > 1.0:
                        confidence = confidence / 100.0
                except:
                    pass
            elif line.startswith('Reasoning:'):
                reasoning = line.split('Reasoning:')[1].strip()
        
        # Return match if valid choice
        if match_num > 0 and match_num <= len(top_candidates):
            matched_id = top_candidates[match_num - 1][2]
            return matched_id, confidence, cost, reasoning
        
        return None, confidence, cost, reasoning
        
    except Exception as e:
        return None, 0.0, 0.0, f"Error: {str(e)}"



def llm_matching_improved(df_a: pd.DataFrame,
                          df_b: pd.DataFrame, 
                          id_col_a: str,
                          id_col_b: str,
                          client,
                          confidence_threshold: float = 0.5,
                          top_k: int = 20,
                          blocking_threshold: float = 0.1,
                          max_text_length: int = 500,
                          save_details: bool = False) -> Tuple[Set[Tuple[str, str]], float, Optional[pd.DataFrame]]:
    """
    Full improved LLM matching with confidence threshold optimization.
    
    Args:
        df_a: Dataset A
        df_b: Dataset B
        id_col_a: ID column in dataset A
        id_col_b: ID column in dataset B
        client: OpenAI client
        confidence_threshold: Minimum confidence to predict match (default: 0.5)
        top_k: Number of candidates per query (default: 20)
        blocking_threshold: Minimum JW similarity (default: 0.1)
        max_text_length: Max text length (default: 500, 0 = unlimited)
        save_details: Whether to return detailed results DataFrame
    
    Returns:
        Tuple of (predicted_matches, total_cost, details_df)
    """
    predicted_matches = set()
    total_cost = 0.0
    details = []
    
    print(f"\n🚀 Running IMPROVED LLM matching with:")
    print(f"   • Top-K: {top_k} (baseline: 10)")
    print(f"   • Blocking threshold: {blocking_threshold} (baseline: 0.3)")
    print(f"   • Max text length: {max_text_length} chars (baseline: 100)")
    print(f"   • Confidence threshold: {confidence_threshold}")
    print(f"   • Enhanced prompt: ✓")
    print()
    
    for idx_a, row_a in tqdm(df_a.iterrows(), total=len(df_a), desc="Improved LLM matching"):
        matched_id, confidence, cost, reasoning = llm_match_improved(
            row_a, df_b, id_col_b, client, top_k, blocking_threshold, max_text_length
        )
        
        total_cost += cost
        
        # IMPROVEMENT 5: Use confidence threshold
        if matched_id and confidence >= confidence_threshold:
            predicted_matches.add((str(row_a[id_col_a]), str(matched_id)))
        
        if save_details:
            details.append({
                'id_a': str(row_a[id_col_a]),
                'text_a': row_a['text'][:100] + "...",
                'matched_id_b': str(matched_id) if matched_id else None,
                'confidence': confidence,
                'cost': cost,
                'reasoning': reasoning[:200] if reasoning else None,
                'predicted': matched_id is not None and confidence >= confidence_threshold
            })
    
    details_df = pd.DataFrame(details) if save_details else None
    
    return predicted_matches, total_cost, details_df



def optimize_llm_threshold(df_a: pd.DataFrame,
                           df_b: pd.DataFrame,
                           true_matches: Set[Tuple[str, str]],
                           id_col_a: str,
                           id_col_b: str,
                           client,
                           details_df: pd.DataFrame,
                           method_name: str = "Improved-LLM") -> Dict:
    """
    Optimize confidence threshold for improved LLM matching.
    
    Since we already ran the LLM and have confidence scores, we can
    optimize the threshold without re-running (saves cost!).
    
    Args:
        df_a, df_b: Datasets
        true_matches: Ground truth matches
        id_col_a, id_col_b: ID columns
        client: OpenAI client (not used here, but kept for consistency)
        details_df: DataFrame with confidence scores from previous run
        method_name: Name for display
    
    Returns:
        Dictionary with best threshold and metrics
    """
    print(f"\n🔍 Optimizing confidence threshold for {method_name}...")
    print("(Using cached LLM responses - no additional API calls!)")
    
    thresholds = np.arange(0.0, 1.01, 0.05)
    best_threshold = 0.5
    best_f1 = 0.0
    results = []
    
    total_pairs = len(df_a) * len(df_b)
    
    for threshold in thresholds:
        # Build predicted matches using this threshold
        predicted = set()
        for _, row in details_df.iterrows():
            if row['matched_id_b'] is not None and row['confidence'] >= threshold:
                predicted.add((row['id_a'], row['matched_id_b']))
        
        # Calculate metrics
        tp = len(predicted & true_matches)
        fp = len(predicted - true_matches)
        fn = len(true_matches - predicted)
        tn = total_pairs - tp - fp - fn
        
        precision = tp / len(predicted) if len(predicted) > 0 else 0.0
        recall = tp / len(true_matches) if len(true_matches) > 0 else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        
        results.append({
            'threshold': threshold,
            'f1': f1,
            'precision': precision,
            'recall': recall,
            'predicted_count': len(predicted)
        })
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    
    results_df = pd.DataFrame(results)
    best_row = results_df[results_df['threshold'] == best_threshold].iloc[0]
    
    print(f"\n✅ Best threshold: {best_threshold:.2f}")
    print(f"   F1: {best_row['f1']:.3f}")
    print(f"   Precision: {best_row['precision']:.3f}")
    print(f"   Recall: {best_row['recall']:.3f}")
    print(f"   Predicted matches: {best_row['predicted_count']:.0f}")
    
    return {
        'best_threshold': best_threshold,
        'best_f1': best_f1,
        'best_precision': best_row['precision'],
        'best_recall': best_row['recall'],
        'threshold_results': results_df
    }


print("LLM matching ready")


Jaro-Winkler ready
Levenshtein ready
Monge-Elkan ready
Soft-TFIDF ready
TF-IDF ready
Sentence Transformer matching ready
OpenaAI Embedding matching ready
LLM matching ready


## Top-1 Matching & Evaluation

In [18]:
def top1_matching(df_a, df_b, similarity_function, threshold, id_col_a, id_col_b):
    """
    Top-1 matching strategy.
    """
    predicted_matches = set()
    
    for idx_a, row_a in tqdm(df_a.iterrows(), total=len(df_a), desc="Matching"):
        best_sim = 0.0
        best_match_id = None
        
        for idx_b, row_b in df_b.iterrows():
            sim = similarity_function(row_a['text'], row_b['text'])
            if sim > best_sim:
                best_sim = sim
                best_match_id = row_b[id_col_b]
        
        if best_sim >= threshold:
            predicted_matches.add((str(row_a[id_col_a]), str(best_match_id)))
    
    return predicted_matches

def calculate_metrics(predicted_matches, true_matches, total_possible_pairs):
    """
    Calculate evaluation metrics.
    """
    tp = len(predicted_matches & true_matches)
    fp = len(predicted_matches - true_matches)
    fn = len(true_matches - predicted_matches)
    tn = total_possible_pairs - tp - fp - fn
    
    precision = tp / len(predicted_matches) if len(predicted_matches) > 0 else 0.0
    recall = tp / len(true_matches) if len(true_matches) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    accuracy = (tp + tn) / total_possible_pairs if total_possible_pairs > 0 else 0.0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'true_positives': tp,
        'false_positives': fp,
        'false_negatives': fn
    }

def find_best_threshold(df_a, df_b, similarity_function, true_matches,
                        id_col_a, id_col_b, method_name):
    """
    Find optimal threshold.
    """
    print(f"\n🔍 Optimizing threshold for {method_name}...")
    
    thresholds = np.arange(0.50, 0.96, 0.05)
    best_threshold = 0.0
    best_f1 = 0.0
    total_pairs = len(df_a) * len(df_b)
    
    for threshold in thresholds:
        predicted = top1_matching(df_a, df_b, similarity_function,
                                  threshold, id_col_a, id_col_b)
        metrics = calculate_metrics(predicted, true_matches, total_pairs)
        
        if metrics['f1'] > best_f1:
            best_f1 = metrics['f1']
            best_threshold = threshold
        
        print(f"  Threshold {threshold:.2f}: F1={metrics['f1']:.3f}")
    
    print(f"\n✅ Best: {best_threshold:.2f} (F1={best_f1:.3f})")
    return best_threshold

print("Matching and evaluation functions ready!")

Matching and evaluation functions ready!


## Run All Experiments

In [20]:
all_results = []

datasets = {
    'Original': (df_a, df_b),
    'Ciphered': (df_a_ciphered, df_b_ciphered),
    'Scrambled': (df_a_scrambled, df_b_scrambled)
}

string_methods = {
    'Jaro-Winkler': jaro_winkler_similarity,
    'Levenshtein': levenshtein_similarity,
    'Monge-Elkan': monge_elkan_similarity,
    'Soft-TFIDF': soft_tfidf_similarity
}

print("\n" + "="*80)
print("STARTING ALL EXPERIMENTS")
print("="*80)
print(f"Total experiments: {(len(string_methods) + 2) * 3}")
if OPENAI_API_KEY:
    print(f"   + 6 more with OpenAI methods = {(len(string_methods) + 2) * 3 + 6}")
print("="*80)

experiment_num = 1

# String methods
for method_name, similarity_func in string_methods.items():
    for dataset_name, (data_a, data_b) in datasets.items():
        
        print(f"\n[{experiment_num}] {method_name} on {dataset_name}")
        print("="*80)
        
        start_time = time.time()
        
        best_threshold = find_best_threshold(
            data_a, data_b, similarity_func, true_matches,
            CONFIG['id_col_a'], CONFIG['id_col_b'], method_name
        )
        
        predicted = top1_matching(
            data_a, data_b, similarity_func,
            best_threshold, CONFIG['id_col_a'], CONFIG['id_col_b']
        )
        
        metrics = calculate_metrics(predicted, true_matches, len(data_a) * len(data_b))
        exec_time = time.time() - start_time
        
        all_results.append({
            'Method': method_name,
            'Dataset': dataset_name,
            'Threshold': best_threshold,
            'F1': metrics['f1'],
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'Accuracy': metrics['accuracy'],
            'Time (s)': exec_time,
            'Cost ($)': 0.0
        })
        
        print(f"F1={metrics['f1']:.3f}, Time={exec_time:.1f}s")
        experiment_num += 1

# TF-IDF
for dataset_name, (data_a, data_b) in datasets.items():
    print(f"\n[{experiment_num}] TF-IDF on {dataset_name}")
    print("="*80)
    
    start_time = time.time()
    best_threshold = 0.0
    best_f1 = 0.0
    total_pairs = len(data_a) * len(data_b)
    
    for threshold in np.arange(0.50, 0.96, 0.05):
        predicted = tfidf_matching(data_a, data_b, threshold,
                                  CONFIG['id_col_a'], CONFIG['id_col_b'])
        metrics = calculate_metrics(predicted, true_matches, total_pairs)
        if metrics['f1'] > best_f1:
            best_f1 = metrics['f1']
            best_threshold = threshold
    
    predicted = tfidf_matching(data_a, data_b, best_threshold,
                              CONFIG['id_col_a'], CONFIG['id_col_b'])
    metrics = calculate_metrics(predicted, true_matches, total_pairs)
    exec_time = time.time() - start_time
    
    all_results.append({
        'Method': 'TF-IDF',
        'Dataset': dataset_name,
        'Threshold': best_threshold,
        'F1': metrics['f1'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'Accuracy': metrics['accuracy'],
        'Time (s)': exec_time,
        'Cost ($)': 0.0
    })
    
    print(f"F1={metrics['f1']:.3f}, Time={exec_time:.1f}s")
    experiment_num += 1

# SentenceTransformer
for dataset_name, (data_a, data_b) in datasets.items():
    print(f"\n[{experiment_num}] SentenceTransformer on {dataset_name}")
    print("="*80)
    
    start_time = time.time()
    best_threshold = 0.0
    best_f1 = 0.0
    total_pairs = len(data_a) * len(data_b)
    
    for threshold in np.arange(0.50, 0.96, 0.05):
        predicted = sentence_transformer_matching(data_a, data_b, threshold,
                                                 CONFIG['id_col_a'], CONFIG['id_col_b'])
        metrics = calculate_metrics(predicted, true_matches, total_pairs)
        if metrics['f1'] > best_f1:
            best_f1 = metrics['f1']
            best_threshold = threshold
    
    exec_time = time.time() - start_time
    
    all_results.append({
        'Method': 'SentenceTransformer',
        'Dataset': dataset_name,
        'Threshold': best_threshold,
        'F1': best_f1,
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'Accuracy': metrics['accuracy'],
        'Time (s)': exec_time,
        'Cost ($)': 0.0
    })
    
    print(f"✅ F1={best_f1:.3f}, Time={exec_time:.1f}s")
    experiment_num += 1

# OpenAI methods 
if OPENAI_API_KEY:
    # OpenAI Embeddings
    for dataset_name, (data_a, data_b) in datasets.items():
        print(f"\n[{experiment_num}] OpenAI Embeddings on {dataset_name}")
        print("="*80)
        
        start_time = time.time()
        best_threshold = 0.0
        best_f1 = 0.0
        
        for threshold in np.arange(0.50, 0.96, 0.05):
            predicted = openai_embedding_matching(data_a, data_b, threshold,
                                                 CONFIG['id_col_a'], CONFIG['id_col_b'])
            metrics = calculate_metrics(predicted, true_matches, len(data_a) * len(data_b))
            if metrics['f1'] > best_f1:
                best_f1 = metrics['f1']
                best_threshold = threshold
        
        exec_time = time.time() - start_time
        est_cost = (len(data_a) + len(data_b)) * 0.00002
        
        all_results.append({
            'Method': 'OpenAI-Embedding',
            'Dataset': dataset_name,
            'Threshold': best_threshold,
            'F1': best_f1,
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'Accuracy': metrics['accuracy'],
            'Time (s)': exec_time,
            'Cost ($)': est_cost
        })
        
        print(f"✅ F1={best_f1:.3f}, Cost=${est_cost:.4f}")
        experiment_num += 1
    
    # LLM
    for dataset_name, (data_a, data_b) in datasets.items():
        print(f"\n[{experiment_num}] LLM (GPT-4o-mini) on {dataset_name}")
        print("="*80)
        
        start_time = time.time()
        predicted, llm_cost, details_df = llm_matching_improved(
            data_a, data_b,
            CONFIG['id_col_a'], CONFIG['id_col_b'],
            client,
            confidence_threshold=0.5,
            top_k=20,
            blocking_threshold=0.1,
            max_text_length=500,
            save_details=True
        )
        
        # Optimize threshold
        optimization_results = optimize_llm_threshold(
            data_a, data_b, true_matches,
            CONFIG['id_col_a'], CONFIG['id_col_b'],
            client, details_df,
            f"Improved-LLM-{dataset_name}"
        )
        
        best_threshold = optimization_results['best_threshold']
        
        # Get optimized predictions  
        predicted = set()
        for _, row in details_df.iterrows():
            if row['matched_id_b'] is not None and row['confidence'] >= best_threshold:
                predicted.add((row['id_a'], row['matched_id_b']))
        
        llm_cost = llm_cost  # Keep original cost
        
        metrics = calculate_metrics(predicted, true_matches, len(data_a) * len(data_b))
        exec_time = time.time() - start_time
        
        all_results.append({
            'Method': 'LLM-GPT4o-mini',
            'Dataset': dataset_name,
            'Threshold': best_threshold,
            'F1': metrics['f1'],
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'Accuracy': metrics['accuracy'],
            'Time (s)': exec_time,
            'Cost ($)': llm_cost
        })
        
        print(f"✅ F1={metrics['f1']:.3f}, Cost=${llm_cost:.4f}")
        experiment_num += 1

print("\n" + "="*80)
print("ALL EXPERIMENTS COMPLETE!")
print("="*80)


STARTING ALL EXPERIMENTS
Total experiments: 18
   + 6 more with OpenAI methods = 24

[1] Jaro-Winkler on Original

🔍 Optimizing threshold for Jaro-Winkler...


Matching:   1%|          | 13/1081 [00:04<06:41,  2.66it/s]


KeyboardInterrupt: 

## 📊 Step 9: Results Analysis

In [27]:
results_df = pd.DataFrame(all_results)
results_df_sorted = results_df.sort_values('F1', ascending=False)

print("\n" + "="*100)
print("📊 COMPLETE RESULTS")
print("="*100)
display(results_df_sorted)

# Performance drops
print("\n" + "="*100)
print("📉 PERFORMANCE DROP ANALYSIS")
print("="*100)

for method in results_df['Method'].unique():
    print(f"\n{method}:")
    method_results = results_df[results_df['Method'] == method]
    
    orig = method_results[method_results['Dataset'] == 'Original']['F1'].values
    ciph = method_results[method_results['Dataset'] == 'Ciphered']['F1'].values
    scra = method_results[method_results['Dataset'] == 'Scrambled']['F1'].values
    
    if len(orig) > 0:
        print(f"  Original:   {orig[0]:.3f}")
        if len(ciph) > 0:
            drop = ((orig[0] - ciph[0]) / orig[0]) * 100 if orig[0] > 0 else 0
            print(f"  Ciphered:   {ciph[0]:.3f}  (↓ {drop:.1f}%)")
        if len(scra) > 0:
            drop = ((orig[0] - scra[0]) / orig[0]) * 100 if orig[0] > 0 else 0
            print(f"  Scrambled:  {scra[0]:.3f}  (↓ {drop:.1f}%)")

# Best methods
print("\n" + "="*100)
print("🏆 BEST METHOD PER DATASET")
print("="*100)

for dataset in ['Original', 'Ciphered', 'Scrambled']:
    dataset_results = results_df[results_df['Dataset'] == dataset]
    best = dataset_results.loc[dataset_results['F1'].idxmax()]
    print(f"\n{dataset}: {best['Method']} (F1={best['F1']:.3f})")

NameError: name 'all_results' is not defined

## 💾 Step 10: Save Results

In [ ]:
mode_suffix = "_TEST" if TEST_MODE else "_FULL"
output_file = f"results_{CONFIG['dataset_name']}{mode_suffix}.csv"

results_df.to_csv(output_file, index=False)

print(f"\n💾 Results saved to: {output_file}")
print(f"\n✅ Next steps:")
if TEST_MODE:
    print(f"   1. Review the results above")
    print(f"   2. If everything looks good, set TEST_MODE = False")
    print(f"   3. Run the full experiment (~3-4 hours)")
else:
    print(f"   1. Share results with your professor")
    print(f"   2. Run on other datasets if needed")
    print(f"   3. Create visualizations")

## 🎯 Summary

### ✅ What You Have:
- Complete implementation of 8 matching methods
- Tested on 3 dataset versions (Original, Ciphered, Scrambled)
- Full evaluation metrics (F1, Precision, Recall, Accuracy)
- Performance drop analysis
- Results saved to CSV

### 📊 Key Findings:
- Review the results table above
- Compare Original vs Ciphered vs Scrambled
- Identify which methods are most robust

### 🚀 Next Steps:
1. If TEST_MODE: Review results, then run full experiment
2. If FULL_MODE: Share results with professor
3. Replicate on other datasets (amazon-google, dblp-acm, dblp-scholar)

---

**Great work!** 🎉